# BEC and Thermal Cloud NN

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
from generate_bec_thermal_cloud_nn_data import GenerateBecThermalCloudData
from matplotlib.colors import LogNorm

In [ ]:
num_atoms = 100000
#calculate transition temperature

trans_temp = (num_atoms/(2*1*1.645))**0.5
print(trans_temp)

## Generate Training and Test Data

In [ ]:
#training data size, test data size, noise spread, resolution length, number of atoms, transition temperature 

#current temps are {0:87,1:125,2:150,3:280}

#500 noise spread can get 98% accuracy
generate = GenerateBecThermalCloudData(10000,5000,0,100,100000,trans_temp)

#### See What Data Images Look Like

In [ ]:
# T = 170
plt.imshow(generate.x_train[4],cmap = "viridis",norm = LogNorm())

In [ ]:
#plt.scatter(generate.x_train[0][0])
plt.scatter(np.arange(len(generate.x_train[0][0])),generate.x_train[3][50])

In [ ]:
plt.plot(np.arange(len(generate.x_train[0][0])),generate.x_train[0][50],label = "Center of Image")
plt.plot(np.arange(len(generate.x_train[0][0])),generate.x_train[0][45],label = "Edge of Image")
plt.legend(loc='upper left')
plt.xlabel("Image Pixel")
plt.ylabel("Atom Cloud Density")

In [ ]:
N=100000

def getN_ex(temp,trans_temp):
    if temp>=trans_temp:
        return N
    else:
        return N*(temp/trans_temp)**2
    
def getN_o(temp,trans_temp):
    if temp>=trans_temp:
        return 0
    else:
        return N*(1-(temp/trans_temp)**2)

temps=np.arange(trans_temp+100)

N_ex_arr = []
N_o_arr = []
for temp in temps:
    N_ex_arr.append(getN_ex(temp,trans_temp))
    N_o_arr.append(getN_o(temp,trans_temp))

plt.plot(temps,N_ex_arr,label="Thermal Cloud")
plt.plot(temps,N_o_arr,label = "BEC")
plt.legend(loc='center right')
x=[175]
labels = ['T_c']
plt.xticks(x, labels, rotation='horizontal')
plt.xlabel("Temperature")
plt.ylabel("Number of Particles")

## Compile Feed Forward NN

#### Discrete Temperature Values Basic NN

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(4,activation=tf.nn.softmax))

optim = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#### Continuous Temperature Values Basic NN

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5000))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dense(2000))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dense(1))

# optim = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer="adam",loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

#### Discrete Temperature Values Convolutional NN

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(kernel_size=3,filters=12,use_bias=False,padding='same'))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=24,use_bias=False,padding='same',strides=2))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))


model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=32,use_bias=False,padding='same',strides=2))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200,use_bias=False))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(4,activation=tf.nn.softmax))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

x_train = np.asarray(generate.x_train)
x_test = np.asarray(generate.x_test)

x_test = x_test.reshape((x_test.shape[0], 100, 100, 1)).tolist()
x_train = x_train.reshape((x_train.shape[0], 100, 100, 1)).tolist()

#### Continuous Temperature Values Convolutional NN

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(kernel_size=3,filters=12,use_bias=False,padding='same'))
# model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=24,use_bias=False,padding='same',strides=2))
# model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))


model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=32,use_bias=False,padding='same',strides=2))
# model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200,use_bias=False))
# model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation("relu"))

# model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1,activation="relu"))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimizer,loss="mean_squared_error",metrics=["mean_squared_error"])

generate.x_train = np.asarray(generate.x_train)
generate.x_test = np.asarray(generate.x_test)

generate.x_train = generate.x_train.reshape((generate.x_train.shape[0], 100, 100, 1)).tolist()
generate.x_test = generate.x_test.reshape((generate.x_test.shape[0], 100, 100, 1)).tolist()

## Train Model

In [ ]:
# lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * math.pow(0.8,epoch), verbose=True)
# steps_per_epoch = len(generate.x_train)//128
# print(steps_per_epoch)


model.fit(generate.x_train,generate.y_train,epochs=600)#,steps_per_epoch = steps_per_epoch,callbacks = lr_decay_callback)

## Evaluate Results

#### Discrete Accuracy

In [ ]:
val_loss, val_acc = model.evaluate(generate.x_test,generate.y_test)
print("loss: ",val_loss,"\n","accuracy: ",val_acc)
model.evaluate(generate.x_test,generate.y_test)

#### Continuous Accuracy

In [ ]:
val_loss, val_mse = model.evaluate(generate.x_test,generate.y_test)
print("loss: ",val_loss,"\n","mse: ",val_mse)

## See Predictions

#### Continuous Predictions

In [ ]:
#not really a great representation, but you can see the predicted scatterplot over correct scatterplot

predictions = model.predict(generate.x_test)

plt.scatter(np.arange(len(predictions)),predictions)
plt.scatter(np.arange(len(generate.y_test)),generate.y_test)
plt.xlabel("x")
plt.ylabel("y")